In [ ]:
!pip install torchsummary

In [ ]:
!pip install seaborn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms,datasets
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import os
import numpy as np
import cv2
import zipfile
reprocess_data = True
from PIL import Image
from torchsummary import summary
import time
from torch.utils.tensorboard import SummaryWriter
import wandb
import seaborn as sns
import webbrowser


In [ ]:
#Проверка есть ли возможность работать на GPU
torch.cuda.is_available()

In [ ]:
#Автоматический выбор девайса для работы: GPU или CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Смотрим выбранный девайс
print(device)

In [ ]:
#Распаковываем архивы с обучающей и тестовой выборкой
# ! unzip ../input/dogs-vs-cats/test1.zip 
# ! unzip ../input/dogs-vs-cats/train.zip 

In [ ]:
#Присваиваем переменным пути с обучающей и тестовой выборкой
train_path = "./train"
test_path ="./test1"
#Делаем списки из дерикторий
train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

In [ ]:
#Проверяем сколько строк в списках выборок
print(len(train_files))
print(len(test_files))

In [ ]:
#Формируем переменную пути для нулевого файла в обучающей выборке
imgpath = os.path.join(train_path,train_files[0])
print(imgpath)

In [ ]:
#Читаем файл подгруженный в предыдущем шаге и конвертируем его в RGB пространство
img = cv2.imread(imgpath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
type(img)

In [ ]:
#Переводим изображение в многомерную матрицу (3 мерную так как изображение в RGB)
imgtensor = torch.tensor(img)

In [ ]:
#Подгружаем размерность тензора (высота, ширина, глубина) для 0 изображения
imgtensor.shape

In [ ]:
#Объявляем класс с тремя функциями 
class Dataset():
    def __init__(self,filelist,filepath,transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform
    def __len__(self):
        return int(len(self.filelist))
    def __getitem__(self,index):
        imgpath = os.path.join(self.filepath,self.filelist[index])
        img = Image.open(imgpath).convert('L')
        if "dog" in imgpath:
            label = 1
        else:
            label = 0 
        if self.transform is not None:
            img = self.transform(img)
        return (img,label)
            
        

In [ ]:
#Ресайзим изображение до 60px
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])
#Ресайзим обучающую выборку до 60px
train = Dataset(train_files,train_path,transformations)
#Ресайзим тестовую выборку до 60px
test = Dataset(test_files,test_path,transformations)
#Разбиваем обучающую выборку (25к) рандомно на выборку обучения и валидации 20к и 5к
train_set,val_set = torch.utils.data.random_split(train,[20000,5000]) 

In [ ]:
#Проверяем сколько строк в валидационной выборке
len(val_set)

In [ ]:
#Делаем список из валидационной выборки, которую сплитнули на предыдущем шаге
val_set_bal = [val_set.__getitem__(x)[1] for x in range(len(val_set))]

In [ ]:
#Визуализируем валидационную выборку (сколько файлов с собаками(1) и кошками(0))
sns.countplot(val_set_bal)

In [ ]:
print("train set size :",train_set.__len__())
print("val set size :",val_set.__len__())

In [ ]:
#Проверили что 
train_set.__getitem__(0)[0].shape

In [ ]:
#Формируем обучающий и тестовый датасеты по 16 картинок в батче
train_dataset = torch.utils.data.DataLoader(dataset = train,batch_size = 8,shuffle=True)
test_dataset = torch.utils.data.DataLoader(dataset = test,batch_size = 8,shuffle=False)

In [ ]:
val_set.__getitem__(0)

In [ ]:
#Объявляем класс модели
class Covnet(nn.Module):
    def __init__(self):
        super().__init__()
        #Прописываем первый слой
        self.conv1 = nn.Sequential(
            #Объявляем первый сверточный слой где (conv1d - текст; 2d- изображения; 3d - трехмерное изображение)
            #1- слой на вход, в данном случае тензор, 16-выходов, 3-размер ядра
            nn.Conv2d(1,16,3),
            #Функция активации
            nn.ReLU(),
            #Сжимаем с сохранением фичей для следующего слоя
            nn.MaxPool2d(2,2),
            #Боремся с переобучением (убираем 10% нейронов)
            nn.Dropout(0.1)

            ) 
            
        self.conv2 =   nn.Sequential(
            #Аналогично предыдущему, только на вход 16(тк в предыдущем 16 на выходе), 32 - на выходе (16*maxpool), 3-размер ядра(статичен)
            nn.Conv2d(16,32,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.1)

        ) 
        self.conv3 =   nn.Sequential(
            nn.Conv2d(32,64,3),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout(0.1)

            )
        self.fc1 = nn.Sequential(
        #Переводим полученные фичи в вектор
        nn.Flatten(),
        #Создаем линейный слой 1600-входные нейроны (подбором) 256-выходных(степень двойки, подбор)
        nn.Linear(1600,256),
        #Функция активации 
        nn.ReLU(),
        nn.Dropout(0.1)
        )
            
        self.fc2 = nn.Sequential(
        #Создаем финальный линейный слой с 256 нейронами на входе и 2 на выходе (2 тк у нас два класса: коты и собаки)
        nn.Linear(256,2),
        )
                
    def forward(self,x):
        #Соединяем все слои в одно
        x= self.conv1(x)
        x= self.conv2(x)
        x= self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        #Возвращаем все и в конце функция активации softmax для классификации
        return F.softmax(x,dim = 1) 
        


In [ ]:
#Объявляем переменную с классом модели (в шаге выше) 
model = Covnet()
model.to(device)
#Отображаем параметры итоговой модели
summary(model,(1,60,60))

In [ ]:
#Выбираем количество эпох(циклов прогона всех данных через модель)
EPOCHS = 10
#Объявляем переменную оптимизатора адам - по дефолту 0.001, подбирается исходя из результата
optimiser = optim.Adam(model.parameters(),lr=1e-5)
#Используем кросс энтропию(функцию потери) тк она лучше для классификации
LOSS = nn.CrossEntropyLoss()

In [ ]:
print(len(train_set))
print(len(val_set))

In [ ]:
#Объявляем переменную в которую будут суммироваться логи
writer = SummaryWriter()

In [ ]:
for epoch in range(EPOCHS):
    model.train()
    total = 0
    correct = 0
    train_loss = 0
    train_accuracy = 0
    counter = 0
    train_running_loss =0
    with tqdm(train_dataset, unit="batch") as tepoch:
        tepoch.set_description(f'Epoch {epoch+1}/{EPOCHS}')
        for data,label in tepoch:
            data,label = data.to(device) , label.to(device)
            optimiser.zero_grad()
            output = model(data)
            loss = LOSS(output,label)
            loss.backward()
            optimiser.step() 
            train_running_loss+= loss.item() * data.size(0)
            _,pred = torch.max(output.data,1)
            total += label.size(0)
            correct += (pred == label).sum().item()
        train_accuracy = correct/total
        train_loss = train_running_loss/len(train_set)
#         tepoch.set_postfix(accuracy =train_accuracy,loss = train_loss/len(train_dataset))
        print("=====/train/=====")
        print("epoch {} train accuracy {}".format(epoch+1,train_accuracy))
        print("epoch {} train loss {}".format(epoch+1,train_loss))
        writer.add_scalar("train accuracy",train_accuracy,epoch+1)
        writer.add_scalar("train loss",train_loss,epoch+1)
#         wandb.log({"train accuracy":train_accuracy,"train loss":train_loss,"epochs":epoch+1},step = epoch+1)
    if epoch %1 == 0:
        model.eval()
        total = 0
        correct = 0
        val_loss = 0
        val_accuarcy =0
        val_running_loss = 0
        with torch.no_grad():
            for val_data,val_label in test_dataset:
                val_data,val_label = val_data.to(device),val_label.to(device)
                val_output = model(val_data)
                #loss function used is CrossEntropyLoss 
                loss_val = LOSS(val_output,val_label)
                #calacuate the running loss by multiplying loss value by batch size
                val_running_loss+= loss_val.item() * val_data.size(0)
#                 val_running_loss+= loss_val.item() 
                _,pred = torch.max(val_output.data,1)    
                total += val_label.size(0)
                correct += (pred == val_label).sum().item()
            val_accuarcy = correct/total
            #calcuate loss per epoch by dividing runing loss by number of items in validation set
            val_loss = val_running_loss/len(val_set)
            print(val_running_loss)
            print("=====/val/=====")
            print("epoch {} val accuracy {}".format(epoch+1,val_accuarcy))
            print("epoch {} val loss {}".format(epoch+1,val_loss))
            writer.add_scalar("val accuracy",val_accuarcy,epoch+1)
            writer.add_scalar("val loss",val_loss,epoch+1)
#             wandb.log({"val accuracy":val_accuarcy,"val loss":val_loss},step = epoch+1)

In [ ]:
batchsize = 16
=====/train/=====
epoch 10 train accuracy 0.62644
epoch 10 train loss 0.8090380088090897
8183.406176567078
=====/val/=====
epoch 10 val accuracy 0.58624
epoch 10 val loss 1.6366812353134155

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
wandb.finish()